In [18]:
#General parameters & imports
from sympy import *
import math
Vdd=1.2 #V

#vco parameters
Kvco = 300*10**6 #MHz/V

#PDF + Charge Pump
C1, R1 = symbols("C1 R1")

#page 646 RFM Ravazi
#PFD components
FreqIn = 40*10**6 #Hz
Win = 2*math.pi*FreqIn #rad/s

#charge pump components
#Ip, R1, C1, C2
Ip = 500*10**(-6) #A

#to start off, zeta = 1 and 2.5Wn = Win/10
zeta = 1 #unit-less?
Wn = Win/25 #rad/s

#2.5Wn is loop bandwidth (eq 9.57, page 645)
zeta_eq = ((R1)/2)*sqrt((Ip*C1*Kvco)/(2*math.pi)) -1*zeta #eq 9.60
Wn_eq = sqrt((Ip*Kvco)/(2*math.pi*C1)) - 1*Wn #eq 9.91

##Make sure that C1 is manufacturable on-chip, if not change Ip
#from there, let us define derive C1 from Wn
C1_val = solve(Wn_eq, C1)[0]
#not a must, but guarantees condition 9.41 
C2_val = 0.2*C1_val
#finally, this leads to R1 value
R1_val = solve(zeta_eq.subs(C1, C1_val), R1)[0]

print("Loop bandwidth is ", 2.5*Wn)
print("R1 is ", R1_val)
print("C1 is ", C1_val)
print("C2 is ", C2_val)

#Transistors equations
#TODO: Update parameters for TSMC 65
#Parameters for [TSMC 65nm process]
Eox = 34.53 #pF/m
Tox = 7.6 #nm

WL_ratio, VGS, VSB = symbols("WL_ratio VGS VSB")

#PFET
Kpp = 58*10**-6 #A/V^2
VTH0p = -0.65 #V
Gammap = -0.40 # V^1/2
PsiFp = 0.42 #V
Lambdapp = 0.06*10**-6 #m/V

VTHp_eq = VTH0p + Gammap*((abs(2*PsiFp + VSB))**(1/2) - (abs(2*PsiFp))**(1/2))
Idsp_eq = Kpp*WL_ratio*(VGS - VTHp_eq.subs(VSB, 0))**2 

#NFET
Kpn = 170*10**-6 #A/V^2
VTH0n = 0.50 #V
Gamman = 0.58 # V^1/2
PsiFn = 0.44 #V
Lambdapn = 0.06*10**-6 #m/V

#Saturation region (Vgs>Vth)
VTHn_eq = VTH0n + Gamman*((abs(2*PsiFn + VSB))**(1/2) - (abs(2*PsiFn))**(1/2))
Idsn_eq = Kpn*WL_ratio*(VGS - VTHn_eq.subs(VSB, 0))**2

#Current mirror for charge pump

#Resistor to set current
Rref = (Vdd-VTHn_eq.subs([(VSB, 0)]))/Ip
print("Rref = ", Rref)

#NCH Transistors, M1, M2, M6, M7
WL1 = solve((Idsn_eq - Ip).subs([(VGS, 0.9)]), WL_ratio)[0]
WL2 = solve((Idsn_eq - Ip).subs([(VGS, 0.9)]), WL_ratio)[0]

#PCH Transistors M3, M4, M5
WL3 = solve((Idsp_eq - Ip).subs([(VGS, 0.9)]), WL_ratio)[0]
WL4 = solve((Idsp_eq - Ip).subs([(VGS, 0.9)]), WL_ratio)[0]
WL5 = solve((Idsp_eq - Ip).subs([(VGS, 0.9)]), WL_ratio)[0]

#Questions: M4 = M5?, M6= M7?
#Other structure to consider for the current mirror?
#How to pick W or L? is L=65nm?
print("M1 W/L ratio  = ", WL1)
print("M3 W/L ratio  = ", WL3)

0.357 178. 0
